In [1]:
import numpy as np
from torch.utils.data import Dataset, DataLoader
import h5py
from torchvision import transforms
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import models
import os
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm_notebook
import cv2

print(torch.__version__)
device = torch.device('cuda:0')

1.2.0


In [2]:
batch_size = 1

transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
inv_normalize = transforms.Normalize([-0.485/0.229, -0.456/0.224, -0.406/0.225], [1/0.229, 1/0.224, 1/0.225])

path = os.path.dirname(os.getcwd())
trainset = torchvision.datasets.CIFAR10(root= path + '/data', train=True, download=True, transform=transform)
Train_dataloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)
testset = torchvision.datasets.CIFAR10(root= path + '/data', train=False, download=True, transform=transform)
Test_dataloader = DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=2)


classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')




Files already downloaded and verified
Files already downloaded and verified


In [3]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

class Vgg16Conv(nn.Module):
    """
    vgg16 convolution network architecture
    """
    def __init__(self, input_channel=3, num_class=10):
        super(Vgg16Conv, self).__init__()
        model_ft = models.vgg16(pretrained=True)
        set_parameter_requires_grad(model_ft, False) 
        #change FC
        self.model_ft = nn.Sequential(*list(model_ft.features.children()))
        self.FC = nn.Linear(8192, num_class)
    
    def forward(self, x):
        # Perform the usual forward pass
        x = self.model_ft(x)
        x = torch.flatten(x, 1)
        x = self.FC(x)
        return F.softmax(x, dim=1)

In [4]:
from torchsummary import summary

_model = Vgg16Conv(num_class = 10)
_model.to(device)
#summary(_model, input_size=(3, 128, 128))

Vgg16Conv(
  (model_ft): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dila

In [5]:
def tensor_to_img(t):
    """Convert normalized tensor in Cuda to cv2 image"""
    unnormalized = inv_normalize(t)
    npimg = np.transpose(unnormalized.cpu().numpy(), (1, 2, 0))
    return npimg

def img_to_cuda_tensor(img):
    tr_img = np.transpose(img, (2, 0, 1))
    t = torch.from_numpy(tr_img)
    t = normalize(t.float())
    return t.to(device)
    

def imshow(img, title):
    """Custom function to display the image using matplotlib"""    
    npimg = tensor_to_img(img)
    #plot the numpy image
    plt.figure(figsize = (4, 4))
    plt.axis("off")
    plt.imshow(npimg)
    plt.title(title)
    plt.show()